In [12]:
import os
from astropy.io import fits
import sunpy.map
#from sunpy.coordinates import NorthOffsetFrame
import astropy.units as u
import numpy as np
from astropy.coordinates import SkyCoord
from sunpy.coordinates import frames
import matplotlib.pyplot as plt
import matplotlib.colors
import glob
from scipy import ndimage
import numpy as np
import sunpy.timeseries
import scipy.ndimage as ndimage
from scipy.ndimage import label
from scipy.optimize import curve_fit
from scipy.interpolate import RegularGridInterpolator

In [28]:

R_sun = 6.96e10

dtr=np.pi/180
rtd=180/np.pi
folder_path = "/home/anu/Desktop/samapti/carrington/synoptic fits files"
save_dir1="/home/anu/Desktop/samapti/carrington/synoptic fits files/South hemisphere"
save_dir2= "/home/anu/Desktop/samapti/carrington/synoptic fits files/North hemisphere"
save_dir3= "/home/anu/Desktop/samapti/carrington/synoptic fits files/image"
save_dir4="/home/anu/Desktop/samapti/carrington/synoptic fits files/TF_image"
files = os.listdir(folder_path)
fits_files = glob.glob(os.path.join(folder_path, '*.fits'))
for file in fits_files:
    longitude=[]
    longitude_rad=[]
    sinlat=[]
    latitude=[]
    print(fits_files)
    with fits.open(file)as f:
        header = f[0].header
        data=f[0].data
        nx=data.shape[1]
        ny=data.shape[0]
        print(f"{nx}  {ny}")
        d_sinlat=2/ny
        dlon_rad=(360/nx)*dtr
        for i in range (ny):
            sin_lat= -1+i*(2/ny)
            Latitude =np.arcsin(sin_lat)*rtd
            Latitude_rad=  (Latitude*dtr)
            latitude.append(Latitude)
            sinlat.append(sin_lat)
        for j in range(nx):
            Longitude = 0+j*(360/nx)
            Longitude_rad=Longitude*dtr 
            longitude.append(Longitude)
            longitude_rad.append(Longitude_rad)
        Flux=(data)*(R_sun**2)*(d_sinlat)*(dlon_rad)
        S= (R_sun**2)*(d_sinlat)*(dlon_rad)
        print(np.nansum(Flux>0))
        print(np.nansum(Flux<0))
        print(S)

        
        
        lon=np.array(longitude)
        lat= np.array(latitude)
        Sinlat=np.array( sinlat)
        x,y=np.meshgrid(lon, lat)
        X,Y=np.meshgrid(lon, Sinlat )
     
        
        fluxthres1=1.5e19
        fluxthres2= -1.5e19
        
        binary_mask=(Flux>fluxthres1)|(Flux<fluxthres2)
        labeled_array, num_features = label(binary_mask)
        totflux=ndimage.sum(Flux,labeled_array, range(1, num_features + 1))
        Totflux= np.zeros_like(Flux)
        for i, val in enumerate(totflux):
            Totflux[labeled_array == (i + 1)] = val
        regionthres1 = 2e20
        regionthres2 = -2e20
        binary_mask2=(Totflux>regionthres1)| (Totflux<regionthres2)
        y_indices, x_indices = np.where(binary_mask2==1)

        Lat=lat[y_indices]
        Lon=lon[x_indices]
        mask1=(Lat<=0)& (Lat>=-90)
        SH_lat=Lat[mask1]
        SH_lon=Lon[mask1]
        mask2= (Lat<=90) & (Lat>=0)
        NH_lat = Lat[mask2]
        NH_lon = Lon[mask2]

        sorted_indices1= np.argsort(SH_lon)

        SHlon_sorted=SH_lon[sorted_indices1]
        SHlat_sorted=SH_lat[sorted_indices1]
        output1=np.column_stack((SHlon_sorted,SHlat_sorted))
        sorted_indices2 = np.argsort(NH_lon)
        NHlon_sorted= NH_lon[sorted_indices2]
        NHlat_sorted = NH_lat[sorted_indices2]
        output2 = np.column_stack((NHlon_sorted, NHlat_sorted))
        base_name = os.path.basename(file).replace('.fits', '')
        txt_name1= f" {base_name}_SH.txt"
        txt1= os.path.join(save_dir1, txt_name1)
        txt_name2= f" {base_name}_NH.txt"
        txt2= os.path.join(save_dir2, txt_name2)
        np.savetxt(txt1,output1)
        np.savetxt(txt2, output2)
        
        
        fig = plt.figure(figsize=(12,6))
        plt.imshow(data, origin="lower", cmap='gray', aspect=2)              #vmax=100, vmin=-100
        image_name= f" {base_name}.jpeg"
        mag_image= os.path.join(save_dir3, image_name)
        plt.savefig(mag_image, dpi=300)
        plt.close(fig)
        
        fig=plt.figure(figsize=(12,6))
        plt.pcolormesh(x,y,Flux, cmap='gray')
        image_name= f" {base_name}_flux.jpeg"
        mag_image= os.path.join(save_dir3, image_name)
        plt.savefig(mag_image, dpi=300)
        plt.close(fig)
        
        
        fig=plt.figure(figsize=(12,6))
        plt.pcolormesh(x,y, binary_mask2, cmap='gray')
        image_name= f" {base_name}_regionmask.jpeg"
        mag_image= os.path.join(save_dir3, image_name)
        plt.savefig(mag_image, dpi=300)
        plt.close(fig)




['/home/anu/Desktop/samapti/carrington/synoptic fits files/hmi.Synoptic_Mr_small.2298.fits', '/home/anu/Desktop/samapti/carrington/synoptic fits files/hmi.Synoptic_Mr_small.2296.fits', '/home/anu/Desktop/samapti/carrington/synoptic fits files/hmi.Synoptic_Mr_small.2297.fits']
720  360
128035
130718
2.34851504148357e+17


/tmp/ipykernel_84783/2747440826.py:101: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  plt.pcolormesh(x,y,Flux, cmap='gray')
/tmp/ipykernel_84783/2747440826.py:109: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  plt.pcolormesh(x,y, binary_mask2, cmap='gray')


['/home/anu/Desktop/samapti/carrington/synoptic fits files/hmi.Synoptic_Mr_small.2298.fits', '/home/anu/Desktop/samapti/carrington/synoptic fits files/hmi.Synoptic_Mr_small.2296.fits', '/home/anu/Desktop/samapti/carrington/synoptic fits files/hmi.Synoptic_Mr_small.2297.fits']
720  360
129067
128319
2.34851504148357e+17
['/home/anu/Desktop/samapti/carrington/synoptic fits files/hmi.Synoptic_Mr_small.2298.fits', '/home/anu/Desktop/samapti/carrington/synoptic fits files/hmi.Synoptic_Mr_small.2296.fits', '/home/anu/Desktop/samapti/carrington/synoptic fits files/hmi.Synoptic_Mr_small.2297.fits']
720  360
127455
130760
2.34851504148357e+17


In [29]:
folder_path3="/home/anu/Desktop/samapti/carrington/synoptic fits files/South hemisphere"
Files = os.listdir(folder_path3)
txt_files = glob.glob(os.path.join(folder_path3, '*.txt'))
RMS=[]

for Sfile in txt_files:
    P=[]
    Q=[]
    print(Sfile)
    with open (Sfile, "r") as infile:
          for line in infile:
            parts=line.split()
            if len(parts)==2:
                fhi=float(parts[0])
                lamda=float(parts[1])
                P.append(fhi)
                Q.append(lamda)
    M=np.array(P)
    def model(lon,p,q1,r1):
        S= (lon*np.pi)/180
        return p+q1*np.sin(S+r1)
    Params, Cov = curve_fit(model,M,Q,method = 'trf')
    print(f"p = {Params[0]}, q1 = {Params[1]}, r1={Params[2]}")
    Y=model(M, *Params)
#print(SY)
    SQ=np.subtract(Q,Y)
#print(SQ)
    mean=np.mean(SQ**2)
#print(Smean)
    rms=np.sqrt(mean)
    print(rms)
    RMS.append(rms)
    #print(RMS)
#print(Srms)
    Error1=model(lon, *Params)+rms
    Error2=model(lon, *Params)-rms
    
    base_name = os.path.basename(Sfile).replace('.txt', '')
    fig = plt.figure(figsize=(12,6))
    plt.plot (lon,model(lon, *Params), color="Red" ,linewidth=1)
    plt.plot(lon,Error1, color="green",linewidth=1)
    plt.plot(lon,Error2, color="blue",linewidth=1)
    plt.pcolormesh(x,y,Flux,cmap='gray')

    plt.xlabel("carington longitutde", color="black", fontsize=14, fontstyle="oblique")
    plt.ylabel(" latitude", color="black", fontsize=14, fontstyle="oblique")
    image_name= f" {base_name}_STF.jpeg"
    mag_image= os.path.join(save_dir4, image_name)
    plt.savefig(mag_image, dpi=300)
    plt.close(fig)

/home/anu/Desktop/samapti/carrington/synoptic fits files/South hemisphere/ hmi.Synoptic_Mr_small.2296_SH.txt
p = -13.620708189461842, q1 = 3.687488019113918, r1=0.19373771236912127
6.95447770142451


/tmp/ipykernel_84783/2171049074.py:43: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  plt.pcolormesh(x,y,Flux,cmap='gray')


/home/anu/Desktop/samapti/carrington/synoptic fits files/South hemisphere/ hmi.Synoptic_Mr_small.2298_SH.txt
p = -13.642706832142006, q1 = 2.549398964812785, r1=1.1935074195491615
4.3660618212325755


/tmp/ipykernel_84783/2171049074.py:43: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  plt.pcolormesh(x,y,Flux,cmap='gray')


/home/anu/Desktop/samapti/carrington/synoptic fits files/South hemisphere/ hmi.Synoptic_Mr_small.2297_SH.txt
p = -15.5592694594943, q1 = 2.4003790914244307, r1=-0.28872570893078076
6.3057359072332995


/tmp/ipykernel_84783/2171049074.py:43: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  plt.pcolormesh(x,y,Flux,cmap='gray')


In [30]:
folder_path3="/home/anu/Desktop/samapti/carrington/synoptic fits files/South hemisphere"
Files = os.listdir(folder_path3)
txt_files = glob.glob(os.path.join(folder_path3, '*.txt'))

for Sfile in txt_files:
    P=[]
    Q=[]
    print(Sfile)
    with open (Sfile, "r") as infile:
          for line in infile:
            parts=line.split()
            if len(parts)==2:
                fhi=float(parts[0])
                lamda=float(parts[1])
                P.append(fhi)
                Q.append(lamda)
    M=np.array(P)
    def model(lon,p,q1,r1,q2,r2):
        S= (lon*np.pi)/180
        return p+q1*np.sin(S+r1)+q2*np.sin(2*S+r2)
    Params2, Cov2 = curve_fit(model,M,Q,method = 'trf')
    print(f"p = {Params2[0]}, q1 = {Params2[1]}, r1={Params2[2]},q2={Params2[3]},r2={Params2[4]}")
    Y2=model(M, *Params2)
#print(SY)
    SQ=np.subtract(Q,Y2)
#print(SQ)
    mean=np.mean(SQ**2)
#print(Smean)
    rms=np.sqrt(mean)
    print(rms)
    RMS.append(rms)
    #print(RMS)
#print(Srms)
    Error3=model(lon, *Params2)+rms
    Error4=model(lon, *Params2)-rms
    
    base_name = os.path.basename(Sfile).replace('.txt', '')
    fig = plt.figure(figsize=(12,6))
    plt.plot (lon,model(lon, *Params2), color="Red" ,linewidth=1)
    plt.plot(lon,Error3, color="green",linewidth=1)
    plt.plot(lon,Error4, color="blue",linewidth=1)
    plt.pcolormesh(x,y,Flux,cmap='gray')

    plt.xlabel("carington longitutde", color="black", fontsize=14, fontstyle="oblique")
    plt.ylabel(" latitude", color="black", fontsize=14, fontstyle="oblique")
    image_name= f" {base_name}_STF2.jpeg"
    mag_image= os.path.join(save_dir4, image_name)
    plt.savefig(mag_image, dpi=300)
    plt.close(fig)

/home/anu/Desktop/samapti/carrington/synoptic fits files/South hemisphere/ hmi.Synoptic_Mr_small.2296_SH.txt
p = -14.155803656314765, q1 = 2.700837494061839, r1=0.13811401725571348,q2=4.134407028358483,r2=-0.43115175789952787
6.437417574958422


/tmp/ipykernel_84783/1226803634.py:42: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  plt.pcolormesh(x,y,Flux,cmap='gray')


/home/anu/Desktop/samapti/carrington/synoptic fits files/South hemisphere/ hmi.Synoptic_Mr_small.2298_SH.txt
p = -12.694438274115424, q1 = 1.8815394728912438, r1=1.0776950602160846,q2=2.537970022794265,r2=0.4715734293806089
4.078285576822813


/tmp/ipykernel_84783/1226803634.py:42: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  plt.pcolormesh(x,y,Flux,cmap='gray')


/home/anu/Desktop/samapti/carrington/synoptic fits files/South hemisphere/ hmi.Synoptic_Mr_small.2297_SH.txt
p = -14.360152799786206, q1 = 1.522034493981605, r1=-0.43757931202515804,q2=4.5374999526243585,r2=-0.2606698968316234
5.72032736897874


/tmp/ipykernel_84783/1226803634.py:42: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  plt.pcolormesh(x,y,Flux,cmap='gray')


In [31]:
folder_path3="/home/anu/Desktop/samapti/carrington/synoptic fits files/South hemisphere"
Files = os.listdir(folder_path3)
txt_files = glob.glob(os.path.join(folder_path3, '*.txt'))

for Sfile in txt_files:
    P=[]
    Q=[]
    print(Sfile)
    with open (Sfile, "r") as infile:
          for line in infile:
            parts=line.split()
            if len(parts)==2:
                fhi=float(parts[0])
                lamda=float(parts[1])
                P.append(fhi)
                Q.append(lamda)
    M=np.array(P)
    def model(lon,p,q1,r1,q2,r2,q3,r3):
        S= (lon*np.pi)/180
        return p+q1*np.sin(S+r1)+q2*np.sin(2*S+r2)+q3*np.sin(3*S+r3)
    Params3, Cov3 = curve_fit(model,M,Q,method = 'trf')
    print(f"p = {Params3[0]}, q1 = {Params3[1]}, r1={Params3[2]},q2={Params3[3]},r2={Params3[4]},q3={Params3[5]},r3={Params3[6]}")
    Y3=model(M, *Params3)
#print(SY)
    SQ=np.subtract(Q,Y3)
#print(SQ)
    mean=np.mean(SQ**2)
#print(Smean)
    rms=np.sqrt(mean)
    print(rms)
    RMS.append(rms)
    
#print(Srms)
    Error5=model(lon, *Params3)+rms
    Error6=model(lon, *Params3)-rms
    
    base_name = os.path.basename(Sfile).replace('.txt', '')
    fig = plt.figure(figsize=(12,6))
    plt.plot (lon,model(lon, *Params3), color="Red" ,linewidth=1)
    plt.plot(lon,Error5, color="green",linewidth=1)
    plt.plot(lon,Error6, color="blue",linewidth=1)
    plt.pcolormesh(x,y,Flux,cmap='gray')

    plt.xlabel("carington longitutde", color="black", fontsize=14, fontstyle="oblique")
    plt.ylabel(" latitude", color="black", fontsize=14, fontstyle="oblique")
    image_name= f" {base_name}_STF3.jpeg"
    mag_image= os.path.join(save_dir4, image_name)
    plt.savefig(mag_image, dpi=300)
    plt.close(fig)

/home/anu/Desktop/samapti/carrington/synoptic fits files/South hemisphere/ hmi.Synoptic_Mr_small.2296_SH.txt
p = -14.564720973026517, q1 = 2.521287352543567, r1=0.038005125847900105,q2=4.2826474613598835,r2=-0.3736705452121418,q3=-1.7935836360229183,r3=1.42137540617683
6.316151546422414


/tmp/ipykernel_84783/1766123186.py:42: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  plt.pcolormesh(x,y,Flux,cmap='gray')


/home/anu/Desktop/samapti/carrington/synoptic fits files/South hemisphere/ hmi.Synoptic_Mr_small.2298_SH.txt
p = -12.188517440140078, q1 = 1.0201631318883961, r1=0.5821904884758617,q2=2.9056816310316598,r2=0.3644590192250535,q3=-1.9646522876895047,r3=0.484321758036731
3.870175320824479


/tmp/ipykernel_84783/1766123186.py:42: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  plt.pcolormesh(x,y,Flux,cmap='gray')


/home/anu/Desktop/samapti/carrington/synoptic fits files/South hemisphere/ hmi.Synoptic_Mr_small.2297_SH.txt
p = -14.739826557930343, q1 = 2.5422594115558184, r1=-0.03462659978084419,q2=4.3770176215914525,r2=-0.0946869795420348,q3=-4.136986580577231,r3=-4.837532905998332
4.908905769844171


/tmp/ipykernel_84783/1766123186.py:42: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  plt.pcolormesh(x,y,Flux,cmap='gray')


In [32]:
folder_path3="/home/anu/Desktop/samapti/carrington/synoptic fits files/South hemisphere"
Files = os.listdir(folder_path3)
txt_files = glob.glob(os.path.join(folder_path3, '*.txt'))

for Sfile in txt_files:
    P=[]
    Q=[]
    print(Sfile)
    with open (Sfile, "r") as infile:
          for line in infile:
            parts=line.split()
            if len(parts)==2:
                fhi=float(parts[0])
                lamda=float(parts[1])
                P.append(fhi)
                Q.append(lamda)
    M=np.array(P)
    def model(lon,p,q1,r1,q2,r2,q3,r3,q4,r4):
        S= (lon*np.pi)/180
        return p+q1*np.sin(S+r1)+q2*np.sin(2*S+r2)+q3*np.sin(3*S+r3)+q4*np.sin(4*S+r4)
    Params4, Cov4 = curve_fit(model,M,Q,method = 'trf')
    print(f"p = {Params4[0]}, q1 = {Params4[1]}, r1={Params4[2]},q2={Params4[3]},r2={Params4[4]},q3={Params4[5]},r3={Params4[6]},q4={Params4[7]},r4={Params4[8]}")
    Y4=model(M, *Params4)
#print(SY)
    SQ=np.subtract(Q,Y4)
#print(SQ)
    mean=np.mean(SQ**2)
#print(Smean)
    rms=np.sqrt(mean)
    print(rms)
    RMS.append(rms)
#print(Srms)
    Error7=model(lon, *Params4)+rms
    Error8=model(lon, *Params4)-rms
    
    base_name = os.path.basename(Sfile).replace('.txt', '')
    fig = plt.figure(figsize=(12,6))
    plt.plot (lon,model(lon, *Params4), color="Red" ,linewidth=1)
    plt.plot(lon,Error7, color="green",linewidth=1)
    plt.plot(lon,Error8, color="blue",linewidth=1)
    plt.pcolormesh(x,y,Flux,cmap='gray')

    plt.xlabel("carington longitutde", color="black", fontsize=14, fontstyle="oblique")
    plt.ylabel(" latitude", color="black", fontsize=14, fontstyle="oblique")
    image_name= f" {base_name}_STF4.jpeg"
    mag_image= os.path.join(save_dir4, image_name)
    plt.savefig(mag_image, dpi=300)
    plt.close(fig)

/home/anu/Desktop/samapti/carrington/synoptic fits files/South hemisphere/ hmi.Synoptic_Mr_small.2296_SH.txt
p = -14.318272525625611, q1 = 2.8615955768833876, r1=0.5265686733457026,q2=2.565191705796273,r2=-0.22935516118135008,q3=2.4713063905071095,r3=4.497878181801849,q4=4.356362914866081,r4=2.50053693699237
5.740915693182704


/tmp/ipykernel_84783/995100289.py:41: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  plt.pcolormesh(x,y,Flux,cmap='gray')


/home/anu/Desktop/samapti/carrington/synoptic fits files/South hemisphere/ hmi.Synoptic_Mr_small.2298_SH.txt
p = -12.227098924707335, q1 = 0.9458091406556289, r1=0.3032111759572474,q2=3.3009513438050075,r2=0.19763157962282396,q3=-2.1701069464086924,r3=0.6099523887574315,q4=2.1664923109330076,r4=3.925866203562318
3.690003967466932


/tmp/ipykernel_84783/995100289.py:41: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  plt.pcolormesh(x,y,Flux,cmap='gray')


/home/anu/Desktop/samapti/carrington/synoptic fits files/South hemisphere/ hmi.Synoptic_Mr_small.2297_SH.txt
p = -15.384848518626026, q1 = 3.2771946703471735, r1=0.16564718743556806,q2=3.053688188697652,r2=-0.19996120687977617,q3=-4.645267845494137,r3=7.901297459497888,q4=-1.3113327913338109,r4=11.67419804280268
4.86976088152604


/tmp/ipykernel_84783/995100289.py:41: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  plt.pcolormesh(x,y,Flux,cmap='gray')


In [33]:
folder_path3="/home/anu/Desktop/samapti/carrington/synoptic fits files/South hemisphere"
Files = os.listdir(folder_path3)
txt_files = glob.glob(os.path.join(folder_path3, '*.txt'))

for Sfile in txt_files:
    P=[]
    Q=[]
    print(Sfile)
    with open (Sfile, "r") as infile:
          for line in infile:
            parts=line.split()
            if len(parts)==2:
                fhi=float(parts[0])
                lamda=float(parts[1])
                P.append(fhi)
                Q.append(lamda)
    M=np.array(P)
    def model(lon,p,q1,r1,q2,r2,q3,r3,q4,r4,q5,r5):
        S= (lon*np.pi)/180
        return p+q1*np.sin(S+r1)+q2*np.sin(2*S+r2)+q3*np.sin(3*S+r3)+q4*np.sin(4*S+r4)+q5*np.sin(5*S+r5)
    Params5, Cov5 = curve_fit(model,M,Q,method = 'trf')
    print(f"p = {Params5[0]}, q1 = {Params5[1]}, r1={Params5[2]},q2={Params5[3]},r2={Params5[4]},q3={Params5[5]},r3={Params5[6]},q4={Params5[7]},r4={Params5[8]},q5={Params5[9]},r5={Params5[10]}")
    Y5=model(M, *Params5)
#print(SY)
    SQ=np.subtract(Q,Y5)
#print(SQ)
    mean=np.mean(SQ**2)
#print(Smean)
    rms=np.sqrt(mean)
    print(rms)
    RMS.append(rms)
    #print(RMS)
#print(Srms)
    Error9=model(lon, *Params5)+rms
    Error10=model(lon, *Params5)-rms
    
    base_name = os.path.basename(Sfile).replace('.txt', '')
    fig = plt.figure(figsize=(12,6))
    plt.plot (lon,model(lon, *Params5), color="Red" ,linewidth=1)
    plt.plot(lon,Error9, color="green",linewidth=1)
    plt.plot(lon,Error10, color="blue",linewidth=1)
    plt.pcolormesh(x,y,Flux,cmap='gray')

    plt.xlabel("carington longitutde", color="black", fontsize=14, fontstyle="oblique")
    plt.ylabel(" latitude", color="black", fontsize=14, fontstyle="oblique")
    image_name= f" {base_name}_STF5.jpeg"
    mag_image= os.path.join(save_dir4, image_name)
    plt.savefig(mag_image, dpi=300)
    plt.close(fig)

/home/anu/Desktop/samapti/carrington/synoptic fits files/South hemisphere/ hmi.Synoptic_Mr_small.2296_SH.txt
p = -14.368401259473481, q1 = 2.6780808415027924, r1=0.6223535769510601,q2=2.3578918282495716,r2=-0.08669279013604075,q3=2.3642052664106235,r3=10.753441611037317,q4=4.173917664052987,r4=2.5899939819647,q5=-0.8737276467435394,r5=5.817967628402257
5.721637966732838


/tmp/ipykernel_84783/3077529693.py:42: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  plt.pcolormesh(x,y,Flux,cmap='gray')


/home/anu/Desktop/samapti/carrington/synoptic fits files/South hemisphere/ hmi.Synoptic_Mr_small.2298_SH.txt
p = -12.326484182826864, q1 = 1.0665665329347012, r1=0.11037615643599369,q2=2.7581346979079595,r2=0.20017250402538603,q3=1.7423228107075703,r3=3.4975049504260545,q4=2.0644317948988475,r4=10.165827170620553,q5=1.7365143413057151,r5=-0.38277857708647833
3.5924836745326707


/tmp/ipykernel_84783/3077529693.py:42: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  plt.pcolormesh(x,y,Flux,cmap='gray')


/home/anu/Desktop/samapti/carrington/synoptic fits files/South hemisphere/ hmi.Synoptic_Mr_small.2297_SH.txt
p = -15.027436110170264, q1 = 2.7922725393456114, r1=-0.04038788732127345,q2=3.3137650309698654,r2=-0.24807585857080858,q3=-3.775998904221132,r3=7.871803312450909,q4=-1.764903183099362,r4=-6.933467219767791,q5=1.833620650510696,r5=0.12242558272224714
4.78262220016165


/tmp/ipykernel_84783/3077529693.py:42: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  plt.pcolormesh(x,y,Flux,cmap='gray')


In [34]:
folder_path3="/home/anu/Desktop/samapti/carrington/synoptic fits files/North hemisphere"
Files = os.listdir(folder_path3)
txt_files = glob.glob(os.path.join(folder_path3, '*.txt'))
RMS=[]

for Nfile in txt_files:
    A=[]
    B=[]
    print(Nfile)
    with open (Nfile, "r") as infile:
          for line in infile:
            parts=line.split()
            if len(parts)==2:
                fhi=float(parts[0])
                lamda=float(parts[1])
                A.append(fhi)
                B.append(lamda)
    N=np.array(A)
    def model(lon,a,b1,c1):
        S= (lon*np.pi)/180
        return a+b1*np.sin(S+c1)
    params1, Cov1 = curve_fit(model,N,B,method = 'trf')
    print(f"a= {params1[0]}, b= {params1[1]}, c={params1[2]}")
    Y1=model(N, *params1)
#print(SY)
    NB=np.subtract(B,Y1)
#print(SQ)
    mean=np.mean(NB**2)
#print(Smean)
    rms=np.sqrt(mean)
    print(rms)
    RMS.append(rms)
    
    
#print(Srms)
    error1=model(lon, *params1)+rms
    error2=model(lon, *params1)-rms
    
    base_name = os.path.basename(Nfile).replace('.txt', '')
    fig = plt.figure(figsize=(12,6))
    plt.plot (lon,model(lon, *params1), color="Red" ,linewidth=1)
    plt.plot(lon,error1, color="green",linewidth=1)
    plt.plot(lon,error2, color="blue",linewidth=1)
    plt.pcolormesh(x,y,Flux,cmap='gray')

    plt.xlabel("carington longitutde", color="black", fontsize=14, fontstyle="oblique")
    plt.ylabel(" latitude", color="black", fontsize=14, fontstyle="oblique")
    image_name= f" {base_name}_NTF.jpeg"
    mag_image= os.path.join(save_dir4, image_name)
    plt.savefig(mag_image, dpi=300)
    plt.close(fig)

/home/anu/Desktop/samapti/carrington/synoptic fits files/North hemisphere/ hmi.Synoptic_Mr_small.2298_NH.txt
a= 12.891356523481008, b= 6.437166737800593, c=1.8041810033617496
4.6873748802407516


/tmp/ipykernel_84783/1038612392.py:44: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  plt.pcolormesh(x,y,Flux,cmap='gray')


/home/anu/Desktop/samapti/carrington/synoptic fits files/North hemisphere/ hmi.Synoptic_Mr_small.2297_NH.txt
a= 12.296025882752899, b= 3.7486130974922665, c=1.1192827140422161
5.229741252622516


/tmp/ipykernel_84783/1038612392.py:44: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  plt.pcolormesh(x,y,Flux,cmap='gray')


/home/anu/Desktop/samapti/carrington/synoptic fits files/North hemisphere/ hmi.Synoptic_Mr_small.2296_NH.txt
a= 12.46449085387577, b= 2.0287535863516313, c=1.7294766593171689
6.7066058765880525


/tmp/ipykernel_84783/1038612392.py:44: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  plt.pcolormesh(x,y,Flux,cmap='gray')


In [35]:
folder_path3="/home/anu/Desktop/samapti/carrington/synoptic fits files/North hemisphere"
Files = os.listdir(folder_path3)
txt_files = glob.glob(os.path.join(folder_path3, '*.txt'))

for Nfile in txt_files:
    A=[]
    B=[]
    print(Nfile)
    with open (Nfile, "r") as infile:
          for line in infile:
            parts=line.split()
            if len(parts)==2:
                fhi=float(parts[0])
                lamda=float(parts[1])
                A.append(fhi)
                B.append(lamda)
    N=np.array(A)
    def model(lon,a,b1,c1,b2,c2):
        S= (lon*np.pi)/180
        return a+b1*np.sin(S+c1)+b2*np.sin(2*S+c2)
    params2, Cov2 = curve_fit(model,N,B,method = 'trf')
    print(f"a= {params2[0]}, b1= {params2[1]}, c1={params2[2]},b2={params2[3]},c2={params2[4]}")
    Y2=model(N, *params2)
#print(SY)
    NB=np.subtract(B,Y2)
#print(SQ)
    mean=np.mean(NB**2)
#print(Smean)
    rms=np.sqrt(mean)
    print(rms)
    RMS.append(rms)
    error3=model(lon, *params2)+rms
    error4=model(lon, *params2)-rms
    
    base_name = os.path.basename(Nfile).replace('.txt', '')
    fig = plt.figure(figsize=(12,6))
    plt.plot (lon,model(lon, *params2), color="Red" ,linewidth=1)
    plt.plot(lon,error3, color="green",linewidth=1)
    plt.plot(lon,error4, color="blue",linewidth=1)
    plt.pcolormesh(x,y,Flux,cmap='gray')

    plt.xlabel("carington longitutde", color="black", fontsize=14, fontstyle="oblique")
    plt.ylabel(" latitude", color="black", fontsize=14, fontstyle="oblique")
    image_name= f" {base_name}_NTF2.jpeg"
    mag_image= os.path.join(save_dir4, image_name)
    plt.savefig(mag_image, dpi=300)
    plt.close(fig)

/home/anu/Desktop/samapti/carrington/synoptic fits files/North hemisphere/ hmi.Synoptic_Mr_small.2298_NH.txt
a= 13.050571429565206, b1= 5.767955023704527, c1=1.9242071638073694,b2=2.2503900352176904,c2=1.6949431777193686
4.5691150857292975


/tmp/ipykernel_84783/1777759595.py:40: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  plt.pcolormesh(x,y,Flux,cmap='gray')


/home/anu/Desktop/samapti/carrington/synoptic fits files/North hemisphere/ hmi.Synoptic_Mr_small.2297_NH.txt
a= 12.246286706167513, b1= 3.3177268288953043, c1=1.0651358076072246,b2=1.1231323196416803,c2=1.562084848025821
5.195553266410898


/tmp/ipykernel_84783/1777759595.py:40: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  plt.pcolormesh(x,y,Flux,cmap='gray')


/home/anu/Desktop/samapti/carrington/synoptic fits files/North hemisphere/ hmi.Synoptic_Mr_small.2296_NH.txt
a= 11.666735741389784, b1= 1.7526177816756074, c1=1.616405917833256,b2=2.3390749601641976,c2=1.9355435510954928
6.558787077356766


/tmp/ipykernel_84783/1777759595.py:40: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  plt.pcolormesh(x,y,Flux,cmap='gray')


In [36]:
folder_path3="/home/anu/Desktop/samapti/carrington/synoptic fits files/North hemisphere"
Files = os.listdir(folder_path3)
txt_files = glob.glob(os.path.join(folder_path3, '*.txt'))

for Nfile in txt_files:
    A=[]
    B=[]
    print(Nfile)
    with open (Nfile, "r") as infile:
          for line in infile:
            parts=line.split()
            if len(parts)==2:
                fhi=float(parts[0])
                lamda=float(parts[1])
                A.append(fhi)
                B.append(lamda)
    N=np.array(A)
    def model(lon,a,b1,c1,b2,c2,b3,c3):
        S= (lon*np.pi)/180
        return a+b1*np.sin(S+c1)+b2*np.sin(2*S+c2)+b3*np.sin(3*S+c3)
    params3, Cov3 = curve_fit(model,N,B,method = 'trf')
    print(f"a= {params3[0]}, b1= {params3[1]}, c1={params3[2]},b2={params3[3]},c2={params3[4]},b3={params3[5]},c3={params3[6]}")
    Y3=model(N, *params3)
#print(SY)
    NB=np.subtract(B,Y3)
#print(SQ)
    mean=np.mean(NB**2)
#print(Smean)
    rms=np.sqrt(mean)
    print(rms)
    RMS.append(rms)
    error5=model(lon, *params3)+rms
    error6=model(lon, *params3)-rms
    
    base_name = os.path.basename(Nfile).replace('.txt', '')
    fig = plt.figure(figsize=(12,6))
    plt.plot (lon,model(lon, *params3), color="Red" ,linewidth=1)
    plt.plot(lon,error5, color="green",linewidth=1)
    plt.plot(lon,error6, color="blue",linewidth=1)
    plt.pcolormesh(x,y,Flux,cmap='gray')

    plt.xlabel("carington longitutde", color="black", fontsize=14, fontstyle="oblique")
    plt.ylabel(" latitude", color="black", fontsize=14, fontstyle="oblique")
    image_name= f" {base_name}_NTF3.jpeg"
    mag_image= os.path.join(save_dir4, image_name)
    plt.savefig(mag_image, dpi=300)
    plt.close(fig)

/home/anu/Desktop/samapti/carrington/synoptic fits files/North hemisphere/ hmi.Synoptic_Mr_small.2298_NH.txt
a= 13.131628080374604, b1= 5.729215996500988, c1=2.022688925309063,b2=2.3792966669022824,c2=1.7223667254891593,b3=0.8865293237640914,c3=5.414986030087276
4.549854211222


/tmp/ipykernel_84783/844103322.py:40: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  plt.pcolormesh(x,y,Flux,cmap='gray')


/home/anu/Desktop/samapti/carrington/synoptic fits files/North hemisphere/ hmi.Synoptic_Mr_small.2297_NH.txt
a= 12.198672414577954, b1= 3.767734516521471, c1=0.85196512573369,b2=0.8970756006965699,c2=1.4783454755356427,b3=1.3599386637102255,c3=2.72411411374336
5.154498905323487


/tmp/ipykernel_84783/844103322.py:40: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  plt.pcolormesh(x,y,Flux,cmap='gray')


/home/anu/Desktop/samapti/carrington/synoptic fits files/North hemisphere/ hmi.Synoptic_Mr_small.2296_NH.txt
a= 11.807426922507485, b1= 2.5148040541867798, c1=0.7615101465865666,b2=3.2245656089334496,c2=1.9129143117525584,b3=-3.6083415777448407,c3=13.14588924704666
6.283627084710424


/tmp/ipykernel_84783/844103322.py:40: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  plt.pcolormesh(x,y,Flux,cmap='gray')


In [37]:
folder_path3="/home/anu/Desktop/samapti/carrington/synoptic fits files/North hemisphere"
Files = os.listdir(folder_path3)
txt_files = glob.glob(os.path.join(folder_path3, '*.txt'))

for Nfile in txt_files:
    A=[]
    B=[]
    print(Nfile)
    with open (Nfile, "r") as infile:
          for line in infile:
            parts=line.split()
            if len(parts)==2:
                fhi=float(parts[0])
                lamda=float(parts[1])
                A.append(fhi)
                B.append(lamda)
    N=np.array(A)
    def model(lon,a,b1,c1,b2,c2,b3,c3,b4,c4):
        S= (lon*np.pi)/180
        return a+b1*np.sin(S+c1)+b2*np.sin(2*S+c2)+b3*np.sin(3*S+c3)+b4*np.sin(4*S+c4)
    params4, Cov4 = curve_fit(model,N,B,method = 'trf')
    print(f"a= {params4[0]}, b1= {params4[1]}, c1={params4[2]},b2={params4[3]},c2={params4[4]},b3={params4[5]},c3={params4[6]},b4={params4[7]},c4={params4[8]}")
    Y4=model(N, *params4)
#print(SY)
    NB=np.subtract(B,Y4)
#print(SQ)
    mean=np.mean(NB**2)
#print(Smean)
    rms=np.sqrt(mean)
    print(rms)
    RMS.append(rms)
    error7=model(lon, *params4)+rms
    error8=model(lon, *params4)-rms
    
    base_name = os.path.basename(Nfile).replace('.txt', '')
    fig = plt.figure(figsize=(12,6))
    plt.plot (lon,model(lon, *params4), color="Red" ,linewidth=1)
    plt.plot(lon,error7, color="green",linewidth=1)
    plt.plot(lon,error8, color="blue",linewidth=1)
    plt.pcolormesh(x,y,Flux,cmap='gray')

    plt.xlabel("carington longitutde", color="black", fontsize=14, fontstyle="oblique")
    plt.ylabel(" latitude", color="black", fontsize=14, fontstyle="oblique")
    image_name= f" {base_name}_NTF4.jpeg"
    mag_image= os.path.join(save_dir4, image_name)
    plt.savefig(mag_image, dpi=300)
    plt.close(fig)

/home/anu/Desktop/samapti/carrington/synoptic fits files/North hemisphere/ hmi.Synoptic_Mr_small.2298_NH.txt
a= 10.570664284024641, b1= 6.334760030743877, c1=1.8126050481167184,b2=3.4575909096303623,c2=1.9008893746240603,b3=2.32654319104069,c3=0.817206491192071,b4=-5.197598773722912,c4=1.1862867187097659
4.1169268691676475


/tmp/ipykernel_84783/1220351035.py:40: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  plt.pcolormesh(x,y,Flux,cmap='gray')


/home/anu/Desktop/samapti/carrington/synoptic fits files/North hemisphere/ hmi.Synoptic_Mr_small.2297_NH.txt
a= 12.013511269748252, b1= 3.4621367923122524, c1=0.8569309749392034,b2=1.378054756209866,c2=1.366025676953439,b3=0.7315127840065623,c3=3.060640713838995,b4=-1.4438493377408532,c4=-0.5036611689830717
5.087077075519467


/tmp/ipykernel_84783/1220351035.py:40: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  plt.pcolormesh(x,y,Flux,cmap='gray')


/home/anu/Desktop/samapti/carrington/synoptic fits files/North hemisphere/ hmi.Synoptic_Mr_small.2296_NH.txt
a= 12.126197287417895, b1= 2.3278232644368915, c1=0.8239037310427191,b2=2.0725700465672756,c2=2.0983080719103255,b3=-2.787611231115997,c3=0.7344289482340064,b4=1.607904430972906,c4=2.073965809009526
6.228253067056508


/tmp/ipykernel_84783/1220351035.py:40: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  plt.pcolormesh(x,y,Flux,cmap='gray')


In [38]:

folder_path3="/home/anu/Desktop/samapti/carrington/synoptic fits files/North hemisphere"
Files = os.listdir(folder_path3)
txt_files = glob.glob(os.path.join(folder_path3, '*.txt'))

for Nfile in txt_files:
    A=[]
    B=[]
 
    with open (Nfile, "r") as infile:
          for line in infile:
            parts=line.split()
            if len(parts)==2:
                fhi=float(parts[0])
                lamda=float(parts[1])
                A.append(fhi)
                B.append(lamda)
    N=np.array(A)
    def model(lon,a,b1,c1,b2,c2,b3,c3,b4,c4,b5,c5):
        S= (lon*np.pi)/180
        return a+b1*np.sin(S+c1)+b2*np.sin(2*S+c2)+b3*np.sin(3*S+c3)+b4*np.sin(4*S+c4)+b5*np.sin(5*S+c5)
    params5, Cov5 = curve_fit(model,N,B,method = 'trf')
    print(f"a= {params5[0]}, b1= {params5[1]}, c1={params5[2]},b2={params5[3]},c2={params5[4]},b3={params5[5]},c3={params5[6]},b4={params5[7]},c4={params5[8]},b5={params5[9]},c5={params5[10]}")
    Y5=model(N, *params5)
#print(SY)
    NB=np.subtract(B,Y5)
#print(SQ)
    mean=np.mean(NB**2)
#print(Smean)
    rms=np.sqrt(mean)
    print(rms)
    RMS.append(rms)
    error9=model(lon, *params5)+rms
    error10=model(lon, *params5)-rms
    
    base_name = os.path.basename(Nfile).replace('.txt', '')
    fig = plt.figure(figsize=(12,6))
    plt.plot (lon,model(lon, *params5), color="Red" ,linewidth=1)
    plt.plot(lon,error9, color="green",linewidth=1)
    plt.plot(lon,error10, color="blue",linewidth=1)
    plt.pcolormesh(x,y,Flux,cmap='gray')

    plt.xlabel("carington longitutde", color="black", fontsize=14, fontstyle="oblique")
    plt.ylabel(" latitude", color="black", fontsize=14, fontstyle="oblique")
    image_name= f" {base_name}_NTF5.jpeg"
    mag_image= os.path.join(save_dir4, image_name)
    plt.savefig(mag_image, dpi=300)
    plt.close(fig)


a= 10.345138098692304, b1= 7.292241153931313, c1=1.6189071508807162,b2=2.426566013334708,c2=2.172488857512894,b3=2.2682217482078273,c3=0.9873163121292826,b4=-5.639679567297586,c4=1.107862754083111,b5=2.497922008814158,c5=0.4371533203028772
3.956175901872551


/tmp/ipykernel_84783/1074132076.py:40: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  plt.pcolormesh(x,y,Flux,cmap='gray')


a= 13.134879531083172, b1= 3.7334375978517396, c1=0.4642010642575796,b2=0.9110983069156764,c2=1.6110865338207254,b3=1.721703510142809,c3=2.8628461534290306,b4=1.9619074751110888,c4=1.6543105996494156,b5=-2.5755645250588994,c5=1.3209989431473925
4.944703543603289


/tmp/ipykernel_84783/1074132076.py:40: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  plt.pcolormesh(x,y,Flux,cmap='gray')


a= 12.295577533467116, b1= 2.853221855019338, c1=1.2522545238808782,b2=3.3366403898392036,c2=2.7417756604197714,b3=1.3585943508198477,c3=2.0480473568829467,b4=1.6387838149723066,c4=2.2144106088502946,b5=6.237896258152032,c5=11.566272383614791
5.380014341611046


/tmp/ipykernel_84783/1074132076.py:40: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  plt.pcolormesh(x,y,Flux,cmap='gray')
